In [23]:
import json
import datetime
import os

KAKAO_TOKEN_SAVED_FILE = "kakao_msg/kakao_token.json"

def load_tokens(filename):
    with open(filename) as fp:
        tokens = json.load(fp)
        
    return tokens

tokens = load_tokens(KAKAO_TOKEN_SAVED_FILE)

In [24]:
KAKAO_FRIEND_TOKEN_SAVED_FILE = "kakao_msg/kakao_friend_token.json"

def save_tokens(filename, tokens):
    with open(filename, "w") as fp:
        json.dump(tokens, fp)

In [29]:
#https://kauth.kakao.com/oauth/authorize?client_id=어쩌구저쩌구&redirect_uri=https://localhost.com&response_type=code&scope=friends,talk_message
#팀관리 - EDITOR 팀원 초대
#동의항목 - 카카오 서비스 내 친구목록 - 이용중동의

import requests

url = "https://kauth.kakao.com/oauth/token"

data = {
    "grant_type": "authorization_code",
    "client_id": "선생님restAPI",
    "redirect_uri": "https://localhost.com",
    "code": ""
}

response = requests.post(url, data=data)

if response.status_code != 200:
    print("에러!!!")
else:
    tokens = response.json()
    print(tokens)
    
save_tokens(KAKAO_FRIEND_TOKEN_SAVED_FILE, tokens)

{'access_token': 'MDcHXyBXpKouBMTdWdD_ackLrZRD3IAA31B5TD60Cj11GwAAAYD1CqX9', 'token_type': 'bearer', 'refresh_token': 'if4IhqSRqWvhco5i9UtxK4ECWlfqrlnofKJfO_VSCj11GwAAAYD1CqX8', 'expires_in': 21599, 'scope': 'talk_message friends', 'refresh_token_expires_in': 5183999}


In [36]:
import requests

tokens = load_tokens(KAKAO_FRIEND_TOKEN_SAVED_FILE)

friends_url = "https://kapi.kakao.com/v1/api/talk/friends"

headers = {
    "Authorization": "Bearer " + tokens['access_token']
}

res = json.loads(requests.get(friends_url, headers=headers).text)

print(type(res))
#print(res)

flist_uuid = []
friends_list = res.get("elements")
#print(friends_list)

fid = friends_list[0].get("uuid")
#print(fid)

for i in range(len(friends_list)):
    flist_uuid.append(friends_list[i].get("uuid"))
    
print(flist_uuid)

<class 'dict'>
['88D5yPnJ_sjk1efe59fu3eXJ_sr9zfrKtg', '88DxxvTN_sb_0-XT4tbj1-XQ_Mv_yPjP_30', '88L0xvfF8cL2x-vY6tzr2u3Z693xxvLF9cLydA', '88DzwfbB8sP22uvS4dPm0-XS4c36zvnJ_s6w', '88f1wvDF98Ty3unb79_q2eHT_8j8y_vM_Hg']


In [35]:
fsend_url = "https://kapi.kakao.com/v1/api/talk/friends/message/default/send"

print(fid)

headers = {
    "Content-Type": "application/x-www-form-urlencoded",
    "Authorization": "Bearer " + tokens['access_token']
}

print('["{}"]'.format(fid))

data = {
    'receiver_uuids': '["{}"]'.format(fid),
    "template_object": json.dumps({
        "object_type": "text",
        "text": "텍스트 영역입니다. 최대 200자 표시 가능합니다.",
        "link": {
            "web_url": "https://developers.kakao.com",
            "mobile_web_url": "https://developers.kakao.com"
        },
        "button_title": "바로 확인"
    })
}

response = requests.post(fsend_url, headers=headers, data=data)
print(response.status_code)
print(response.json())

if response.status_code != 200:
    print("에러!")
else:
    print("카톡 메시지가 간드아!")

88D5yPnJ_sjk1efe59fu3eXJ_sr9zfrKtg
["88D5yPnJ_sjk1efe59fu3eXJ_sr9zfrKtg"]
400
{'msg': 'API limit has been exceeded.', 'rule_id': 389, 'code': -10}
에러!


In [ ]:
def team_loop_msg(fid_list):
    for i in range(len(fid_list)):
        data = {
            'receiver_uuids': '["{}"]'.format(fid_list[i]),
            "template_object": json.dumps({
                "object_type": "text",
                "text": "xxx 회원님들!!! 오늘부터 50% 초특가 폭탄 세일입니다!!!",
                "link": {
                    "web_url": "https://www.google.com/",
                    "mobile_web_url": "https://www.google.com/"
                },
                "button_title": "공지사항"
            })
        }
        
        requests.post(fsend_url, headers=headers, data=data)
    
team_loop_msg(flist_uuid)